### 학습한 데이터의 결과가 신뢰할수 있는가?

- 모델의 타당성을 검증 
> 크로스 벨리데이션을 확인  
- `Cross-Vaildation : 교차 검증`
> - 특정 데이터를 훈련용과 테스트 전용으로 분할  
> - 훈련데이터로 학습  
> - 테스트데이터로 학습의 타당성을 검증  
- 교차 검증방법중에 K분할 `교차 검증(K-fold(cv값) cross vaildation)`
> - 집합 X => A, B, C로 분할
> - A를 테스트, B, C는 훈련 => 정밀도 s1를 구함
> - B를 테스트, C, A는 훈련 => 정밀도 s2를 구함
> - C를 테스트, A, B는 훈련 => 정밀도 s3를 구함
> - s1, s2, s3를 평균내서 최종 정밀도를 구한다 

In [7]:
import pandas as pd
from sklearn import svm, metrics, model_selection
import sklearn.datasets as ml
import random, re

In [10]:
# 아이리스 데이터 로드
#data = ml.load_iris()
#data
data = pd.read_csv( './data/iris.csv' )
data.head(2)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


In [12]:
train = data[ data.columns[:4] ]
label = data[ data.columns[-1] ]
#train, label

In [15]:
clf    = svm.SVC( gamma='auto' )
# cv : fold의 수 => 세트화
scores = model_selection.cross_val_score( clf, train, label, cv=5 )
print( '5세트 각각의 정밀도 ', scores )
print( '5세트 각각의 정밀도 평균', scores.mean() )

5세트 각각의 정밀도  [0.96666667 1.         0.96666667 0.96666667 1.        ]
5세트 각각의 정밀도 평균 0.9800000000000001


### 그리드 서치 방식
- ML의 알고리즘은 매개변수를 가지고 있다
- 매개변수를 적절하게 수정하면 정답률을 상승 시킬수 있다 => 튜닝, 최적화
- **`매개변수 튜닝을 아주 중요한 작업!!`**
- 이것을 일일이(X)-> 자동화 처리 필요
> 그리드 서치 방식
> 각 매개변수를 적당한 범위에 두고, 알아서 변경하면서 가장 성능이 좋을때까지 값을 찾는 방법
> 하이퍼파라미터 튜닝

In [16]:
import sklearn
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [17]:
train_csv = pd.read_csv('./data/train_mnist.csv')
test_csv  = pd.read_csv('./data/t10k_mnist.csv')

In [20]:
train_csv.shape, test_csv.shape

((749, 785), (249, 785))

In [21]:
# 데이터 추출 
train_label = train_csv.iloc[:, 0]
train_data  = train_csv.iloc[:, 1:]
test_label  = test_csv.iloc[:, 0]
test_data   = test_csv.iloc[:, 1:]

In [22]:
print( train_label.shape, train_data.shape, test_label.shape, test_data.shape  )

(749,) (749, 784) (249,) (249, 784)


In [24]:
# 사용 모델 => 분류기 중에 SVC를 사용하겟다 
# 매개변수는 => C, kernel값에 대한 하이퍼파라미터 튜닝을 하겟다
params = [
    {"C":[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'kernel':['linear'] },
    {"C":[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'kernel':['rbf'], 'gamma':[0.001, 0.0001] }
]

In [25]:
# n_jobs : 병렬 계산 수행할 프로세스수 지정 => -1 => 자동으로 알아서 코어수에 맞게 설정
clf = GridSearchCV( svm.SVC(), params, n_jobs=-1, cv=5 )

In [26]:
# 학습
clf.fit(train_data, train_label)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'C': [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'kernel': ['linear']}, {'C': [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
# 결과 확인
clf.best_estimator_

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [28]:
clf.best_score_

0.8678237650200267

In [29]:
# 예측
pre = clf.predict( test_data )
metrics.accuracy_score( pre, test_label )

0.8755020080321285